In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("/content/hospital inpatient discharge.csv")

/tmp/ipython-input-3653277437.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/hospital inpatient discharge.csv")


In [ ]:
# Shape of data
print("Rows, Columns:", df.shape)

# Column info
print(df.info())

# Missing values
print(df.isnull().sum())


Rows, Columns: (45341, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45341 entries, 0 to 45340
Data columns (total 33 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Hospital Service Area                45229 non-null  object 
 1   Hospital County                      45229 non-null  object 
 2   Operating Certificate Number         45229 non-null  float64
 3   Permanent Facility Id                45229 non-null  float64
 4   Facility Name                        45341 non-null  object 
 5   Age Group                            45341 non-null  object 
 6   Zip Code - 3 digits                  44250 non-null  object 
 7   Gender                               45341 non-null  object 
 8   Race                                 45341 non-null  object 
 9   Ethnicity                            45341 non-null  object 
 10  Length of Stay                       45272 non-null  float64
 11  T

In [ ]:
drop_cols = [
    "CCSR Procedure Code",
    "CCSR Procedure Description",
    "Payment Typology 2",
    "Payment Typology 3",
    "Birth Weight"
]
df.drop(columns=drop_cols, inplace=True)


In [ ]:
# Fill numeric columns with median
df["Length of Stay"].fillna(df["Length of Stay"].median(), inplace=True)

# Fill Zip Code (you can also drop if not important)
df["Zip Code - 3 digits"].fillna(df["Zip Code - 3 digits"].mode()[0], inplace=True)

# Fill categorical columns with mode
df["APR Severity of Illness Code"].fillna(df["APR Severity of Illness Code"].mode()[0], inplace=True)
df["APR Severity of Illness Description"].fillna(df["APR Severity of Illness Description"].mode()[0], inplace=True)
df["APR Risk of Mortality"].fillna(df["APR Risk of Mortality"].mode()[0], inplace=True)


/tmp/ipython-input-1397174575.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Length of Stay"].fillna(df["Length of Stay"].median(), inplace=True)
/tmp/ipython-input-1397174575.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value

Predicting “Type of Admission” (Emergency, Elective, Urgent, etc.) based on patient + medical details.

Input Features: Age Group, Gender, Race, Ethnicity, Length of Stay, Diagnosis, Severity, Costs, etc.

Target Variable: Type of Admission

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Make a copy of dataframe
ml_df = df.copy()

# Encode categorical columns
categorical_cols = ml_df.select_dtypes(include=["object"]).columns

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    ml_df[col] = le.fit_transform(ml_df[col].astype(str))
    label_encoders[col] = le


In [ ]:
from sklearn.model_selection import train_test_split

X = ml_df.drop("Type of Admission", axis=1)   # Features
y = ml_df["Type of Admission"]                # Target

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9418899547910464

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.86      0.86      1396
           1       0.97      0.98      0.97      6337
           2       1.00      1.00      1.00       824
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         9
           5       0.70      0.63      0.66       500

    accuracy                           0.94      9069
   macro avg       0.59      0.58      0.58      9069
weighted avg       0.94      0.94      0.94      9069



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
feature_importance = pd.DataFrame({
    "Feature": X.columns,
    "Importance": model.feature_importances_
}).sort_values(by="Importance", ascending=False)

print(feature_importance.head(10))


                             Feature  Importance
24    Emergency Department Indicator    0.284849
15                      APR DRG Code    0.061191
5                          Age Group    0.061123
18               APR MDC Description    0.059583
17                      APR MDC Code    0.056302
13               CCSR Diagnosis Code    0.056185
26                       Total Costs    0.053773
22  APR Medical Surgical Description    0.048253
25                     Total Charges    0.034882
3              Permanent Facility Id    0.034338


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Decision Tree
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)  # you can tune max_depth
dt_model.fit(X_train, y_train)

# Predictions
y_pred_dt = dt_model.predict(X_test)

# Evaluate
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("\nDecision Tree Report:\n", classification_report(y_test, y_pred_dt))


Decision Tree Accuracy: 0.8919395743742419

Decision Tree Report:
               precision    recall  f1-score   support

           0       0.63      0.92      0.75      1396
           1       0.97      0.94      0.95      6337
           2       1.00      1.00      1.00       824
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00         9
           5       0.50      0.01      0.03       500

    accuracy                           0.89      9069
   macro avg       0.52      0.48      0.45      9069
weighted avg       0.89      0.89      0.87      9069



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
